In [1]:
import sys
sys.path.insert(0, '/home/ons21553/wspace/interview-transcripts/src')
from common import *
from editdist import levenshtein

%load_ext autoreload
%autoreload 2

In [3]:
!which python

/home/ons21553/.local/share/virtualenvs/src-ppTQaxQI/bin/python


In [2]:
import speech_recognition as sr
import recordings.harvard as hv
import pandas as pd

In [57]:
def find_transcript(audio_fpath):
    recognizer = sr.Recognizer()
    
    print(f'\nfinding the best transcript for {audio_fpath}')
    with sr.AudioFile(audio_fpath) as source:
        audio = recognizer.record(source)

    recog_transcript = recognizer.recognize_google(audio)

    all_transcripts = [' '.join(l) for l in hv_data['transcripts']]
    scores = [{
        'ix': i, 
        'text': t, 
        'distance': levenshtein(recog_transcript, t)
    } for i, t in enumerate(all_transcripts)]
    
    scores.sort(key=lambda score: score['distance'])
    
    result = {
        'audio_fpath': audio_fpath,
        **scores[0],
        'next_distance': scores[1]['distance']
    }
    
    print(f'found {result}')
    
    return result

# find_transcript(hv_data['audio']['uk'][0])

In [59]:
hv_data = hv.load()



audio_fpaths = hv_data['audio']['uk'] + hv_data['audio']['us']
results = [find_transcript(audio_fpath) for audio_fpath in audio_fpaths]


finding the best transcript for /home/ons21553/wspace/interview-transcripts/src/../data/recordings/harvard/OSR_uk_000_0049_8k.wav
found {'audio_fpath': '/home/ons21553/wspace/interview-transcripts/src/../data/recordings/harvard/OSR_uk_000_0049_8k.wav', 'ix': 57, 'text': 'It is a band of steel three inches wide. The pipe ran almost the length of the ditch. It was hidden from sight by a mass of leaves and shrubs. The weight. of the package was seen on the high scale. Wake and rise, and step into the green outdoors. The green light in the brown box flickered. The brass tube circled the high wall. The lobes of her ears were pierced to hold rings. Hold the hammer near the end to drive the nail. Next Sunday is the twelfth of the month.', 'distance': 91, 'next_distance': 304}

finding the best transcript for /home/ons21553/wspace/interview-transcripts/src/../data/recordings/harvard/OSR_uk_000_0051_8k.wav
found {'audio_fpath': '/home/ons21553/wspace/interview-transcripts/src/../data/recording

In [64]:
[r for r in results if r['distance']*1.7 >= r['next_distance']]

[]

In [66]:
mapping = {
    r['audio_fpath']: (r['ix'] if r['distance']*1.7 < r['next_distance'] else None)
    for r in results
}

pd.Series(mapping).to_csv(from_data_root('recordings/harvard/mapping.csv'))

/home/ons21553/.local/share/virtualenvs/src-ppTQaxQI/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [13]:
hv_common.MAPPING_FPATH

'/home/ons21553/wspace/interview-transcripts/src/../data/recordings/harvard/mapping.csv'

In [40]:
import os

In [46]:
os.path.basename(r[1]['audio_fpath'])

'OSR_us_000_0017_8k.wav'

In [48]:
mapping = pd.read_csv(hv_common.MAPPING_FPATH, header=None, 
                      names=['audio_fpath', 'transcript_fpath'])
for r in mapping.dropna().iterrows():
    fname = os.path.basename(r[1]['audio_fpath']).replace('.wav', '')
    copyfile(r[1]['audio_fpath'], f'{hv_common.DIR_MATCHED}/{fname}.wav')
    copyfile(r[1]['transcript_fpath'], f'{hv_common.DIR_MATCHED}/{fname}.txt')

In [29]:
list(mapping.iterrows())[0][1].index

Index(['/home/ons21553/wspace/interview-transcripts/src/../data/recordings/harvard/OSR_uk_000_0049_8k.wav', '57'], dtype='object')